# Assignment 1: Dino Fun World

You, in your role as a burgeoning data explorer and visualizer, have been asked by the administrators of a small amusement park in your hometown to answer a couple questions about their park operations. In order to perform the requested analysis, they have provided you with a database containing information about one day of the park's operations.

### Provided Database

The database provided by the park administration is formatted to be readable by any SQL database library. The course staff recommends the sqlite3 library. The database contains three tables, named 'checkins', 'attractions', and 'sequences'. The information contained in each of these tables is listed below:

`checkins`:
    - Description: check-in data for all visitors for the day in the park. The data includes two types of check-ins, inferred and actual checkins.
    - Fields: visitorID, timestamp, attraction, duration, type
`attraction`:
    - The attractions in the park by their corresponding AttractionID, Name, Region, Category, and type. Regions are from the VAST Challenge map such as Coaster Alley, Tundra Land, etc. Categories include Thrill rides, Kiddie Rides, etc. Type is broken into Outdoor Coaster, Other Ride, Carussel, etc.
    - Fields: AttractionID, Name, Region, Category, type
`sequences`:
    - The check-in sequences of visitors. These sequences list the position of each visitor to the park every five minutes. If the visitor has not entered the part yet, the sequence has a value of 0 for that time interval. If the visitor is in the park, the sequence lists the attraction they have most recently checked in to until they check in to a new one or leave the park.
    - Fields: visitorID, sequence
    
The database is named 'dinofunworld.db' and is located in the 'readonly' directory of the Jupyter Notebook environment. It can be accessed at 'readonly/dinofunworld.db'.

### Questions to Answer

The administrators would like you to answer four relatively simple questions about the park activities on the day in question. These questions all deal with park operations and can be answered using the data provided.

Question 1: What is the most popular attraction to visit in the park?
Question 2: What ride (note that not all attractions are rides) has the longest visit time?
Question 3: Which Fast Food offering has the fewest visitors?
Question 4: Compute the Skyline of number of visits and visit time for the park's ride and report the rides that appear in the Skyline.

#### Administrative Notes

This assignment will be graded by Coursera's grading system. In order for your answers to be correctly registered in the system, you must place the code for your answers in the cell indicated for each question. In addition, you should submit the assignment with the output of the code in the cell's display area. The display area should contain only your answer to the question with no extraneous information, or else the answer may not be picked up correctly. Each cell that is going to be graded has a set of comment lines at the beginning of the cell. These lines are extremely important and must not be modified or removed.

In [8]:
# Graded Cell
# PartID:  NDnou
# Question 1: What is the most popular attraction to visit in the park?
# Notes: Your output should be the name of the attraction.
import sqlite3,pandas
db_connect='dinofunworld.db'
conn=sqlite3.connect(db_connect)
pandas.read_sql_query("select Name from (select count(*) total,attraction from checkin group by attraction order by total desc limit 1) a,attraction b where a.attraction=b.AttractionID",conn)

Name
0  Atmosfear

In [6]:
# Graded Cell
# PartID: FXGHp
# Question 2: What ride (note that not all attractions are rides) has the longest average visit time?
# Notes: Your output should be the name of the ride.
import sqlite3,pandas
from datetime import datetime, timedelta
db_connect='dinofunworld.db'
conn=sqlite3.connect(db_connect)
cursor=conn.cursor()
df=pandas.read_sql_query("select attraction,duration from checkin where  attraction in (select AttractionID from attraction where Category like '%Rides%') and duration not like '%@SD' and duration is not null",conn)
df1=pandas.read_sql_query("select count(*) as Totalvisits,attraction  from checkin where  attraction in (select AttractionID from attraction where Category like '%Rides%') and duration not like '%@SD%' and duration is not null group by attraction",conn)
df3=pandas.merge(df, df1, on='attraction')
def convert_to_datetime(time):
    t = datetime.strptime(time,"%H:%M:%S")
    delta = timedelta(hours=t.hour, minutes=t.minute, seconds=t.second)
    return delta.total_seconds()
df['duration'] = df['duration'].apply(convert_to_datetime)
newdf=df.groupby(['attraction']).sum()
new1df=pandas.merge(newdf, df1, on='attraction')
new1df['avg']=round(new1df['duration']/new1df['Totalvisits'],2)
result=new1df.loc[new1df['avg'].idxmax()]
attractiondf=pandas.read_sql_query("select Name,AttractionID from attraction where Category like '%Rides%' ",conn)
listatt=attractiondf.values.tolist()
res = [item for item in listatt
  if item[1] == result.attraction]
res[0][0]

'TerrorSaur'

In [7]:
# Graded Cell
# PartID: KALua
# Question 3: Which Fast Food offering in the park has the fewest visitors?
# Notes: Your output should be the name of the fast food offering.
import sqlite3,pandas
db_connect='dinofunworld.db'
conn=sqlite3.connect(db_connect)
pandas.read_sql_query("select b.Name from (select attraction as attraction,count(*) total from checkin  where  attraction in (select AttractionID  from attraction where type like '%Fast %') group by attraction order by total limit 1) a,attraction b where a.attraction=b.AttractionID",conn)


Name
0  Theresaur Food Stop

In [5]:
# Graded Cell
# PartID: B0LUP
# Question 4: Compute the Skyline of number of visits and visit time for the park's ride and 
#  report the rides that appear in the Skyline. 
# Notes: Remember that in this case, higher visits is better and lower visit times are better. 
#  Your output should be formatted as an array listing the names of the rides in the Skyline.
import sqlite3,pandas
from datetime import datetime, timedelta
db_connect='dinofunworld.db'
conn=sqlite3.connect(db_connect)
cursor=conn.cursor()
df=pandas.read_sql_query("select attraction,duration from checkin where  attraction in (select AttractionID from attraction where Category like '%Rides%') and duration not like '%@SD' and duration is not null",conn)
df1=pandas.read_sql_query("select count(*) as Totalvisits,attraction  from checkin where  attraction in (select AttractionID from attraction where Category like '%Rides%') and duration not like '%@SD%' and duration is not null group by attraction",conn)
df3=pandas.merge(df, df1, on='attraction')
def convert_to_datetime(time):
    t = datetime.strptime(time,"%H:%M:%S")
    delta = timedelta(hours=t.hour, minutes=t.minute, seconds=t.second)
    return delta.total_seconds()
df['duration'] = df['duration'].apply(convert_to_datetime)
newdf=df.groupby(['attraction']).sum()
new1df=pandas.merge(newdf, df1, on='attraction')
new1df['avg']=round(new1df['duration']/new1df['Totalvisits'],2)
result=new1df.loc[new1df['avg'].idxmax()]
attractiondf=pandas.read_sql_query("select Name,AttractionID from attraction where Category like '%Rides%' ",conn)
listatt=attractiondf.values.tolist()
res = [item for item in listatt
  if item[1] == result.attraction]
res[0][0]
list=new1df.values.tolist()
pd=pandas.DataFrame(columns=['attraction'])
finalresult=[]
for  tuple in enumerate(list):
    count=0;
    for  another in enumerate(list):
       if tuple[1][2]<another[1][2] and tuple[1][3]>another[1][3]:
         count=count+1
    if count == 0:
     pd=tuple[1]
     res = [item for item in listatt
      if item[1] == pd[0]]
     finalresult.append(res[0][0])

finalresult

['Wrightiraptor Mountain',
 'Galactosaurus Rage',
 'Keimosaurus Big Spin',
 'Atmosfear',
 'Dykesadactyl Thrill']